In [ ]:
import numpy as np
np.random.seed(1337)  # for reproducibility
import h5py
from sklearn.metrics import roc_curve, auc
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve, average_precision_score

In [ ]:
from keras import backend as K
from keras import optimizers
import tensorflow as tf

## Read Input

In [ ]:
X_0 = np.load('X0.npy')
X_1 = np.load('X1.npy')

X = np.append(X_0,  X_1, axis=0)

In [ ]:
y_1 = np.ones((X_1.shape[0]))
y_0 = np.zeros((X_0.shape[0]))

y = np.append(y_0,  y_1, axis=0)
y = y.reshape(y.shape[0],1)

In [ ]:
X_one_hot = (np.arange(X.max()) == X[...,None]-1).astype(int)

In [ ]:
X_ = X_one_hot.reshape(X_one_hot.shape[0], X_one_hot.shape[1] * X_one_hot.shape[2])

In [ ]:
X_ = X_one_hot.reshape(X_one_hot.shape[0],  X_one_hot.shape[1], X_one_hot.shape[2],1)

## Split Train, Test, Validation

In [ ]:
X_train, X_test, y_train, y_test  = train_test_split(X_, y , test_size=0.15, shuffle = True)
X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.15, shuffle = True)

## MCC metric

In [ ]:
def MCC(y_true, y_pred):
    '''Calculates the Matthews correlation coefficient measure for quality
    of binary classification problems.
    '''
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)

    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())

## CONV2D

In [ ]:
''' Create the model : a network with 3 convolutional layers and a dense layer '''


from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import Input, Convolution2D, MaxPooling2D, Flatten
from keras.models import Model 
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.utils import class_weight


num_classes =1
batch_size = 2014
num_epochs = 150

model = Sequential()
model.add(Conv2D(32, kernel_size=(4,4),activation='relu',input_shape=(299,4,1),padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D((1, 2),padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(64, (4, 4), activation='relu',padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(1, 2),padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(128, (4, 4), activation='relu',padding='same'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(MaxPooling2D(pool_size=(1, 2),padding='same'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(LeakyReLU(alpha=0.1))                  
model.add(Dense(num_classes, activation='sigmoid'))

checkpointer = ModelCheckpoint(filepath='weights.hdf5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='min', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
class_weight = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train.reshape(y_train.shape[0]))



model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy',MCC])

history = model.fit(X_train, y_train,
          epochs=num_epochs,
          batch_size=batch_size,
          validation_data=(X_val, y_val), callbacks=[checkpointer,early], class_weight = class_weight)

## Evaluation

In [ ]:
model.load_weights('weights.hdf5')
fashion_model.compile(loss='binary_crossentropy',
              optimizer= 'adam', metrics=['accuracy',MCC])

score = model.evaluate(X_test, y_test, verbose=0)
score

In [ ]:
np.save('score_CNN_TSS_Start_', score)
np.save('h_DA_MCC_train_', np.asarray(history.history['MCC']))
np.save('h_DA_MCC_val_', np.asarray(history.history['val_MCC']))

In [ ]:
y_pred = fashion_model.predict(X_test)

In [ ]:
np.save('y_pred_CNN_TSS_Start_', y_pred)
np.save('y_test_CNN_TSS_Start_', y_test)

In [ ]:
roc_auc_score(y_test, y_pred)

In [ ]:
average_precision_score(y_test, y_pred)